In [1]:
using System.IO.Pipes;
string pipeName = "ICMNamedPipeServer";
NamedPipeServerStream pipeServer = new NamedPipeServerStream(pipeName, PipeDirection.InOut, 1, PipeTransmissionMode.Message);

In [3]:
using System.Text;
using System.Text.Json;
internal class ExecuteFunction()
{
    /// <summary>
    /// Executes a function on the server by sending a JSON request and receiving a JSON response.
    /// </summary>
    /// <param name="functionName">The name of the function to execute.</param>
    /// <param name="args">The arguments to pass to the function.</param>
    /// <returns>The response from the server as a string.</returns>
    public static object Execute(string functionName, string[] args, NamedPipeServerStream pipeServer)
    {
        if (!System.Runtime.InteropServices.RuntimeInformation.IsOSPlatform(System.Runtime.InteropServices.OSPlatform.Windows))
        {
            return "Platform not supported. Windows 11 is a supported platform";
        }

        string jsonRequest = CreateJsonRequest(functionName, args);
        WriteToServer(jsonRequest);
        string response = ReadFromServer();
        return ParseResponse(response);
    }

    /// <summary>
    /// Creates a JSON request string with the function name and arguments.
    /// </summary>
    /// <param name="functionName">The name of the function to execute.</param>
    /// <param name="args">The arguments to pass to the function.</param>
    /// <returns>The JSON request string with an appended delimiter.</returns>
    private static string CreateJsonRequest(string functionName, string[] args)
    {
        var dict = new Dictionary<string, object>
            {
                { "function", functionName },
                { "args", args }
            };
        string jsonString = JsonSerializer.Serialize(dict);
        jsonString += "\0END\0";
        return jsonString;
    }

    /// <summary>
    /// Writes the JSON request string to the server.
    /// </summary>
    /// <param name="jsonRequest">The JSON request string to send to the server.</param>
    private static void WriteToServer(string jsonRequest)
    {
        pipeServer.Write(Encoding.UTF8.GetBytes(jsonRequest));
        pipeServer.Flush();
    }

    /// <summary>
    /// Reads the response from the server, handling responses larger than the buffer size.
    /// </summary>
    /// <returns>The response string from the server without the delimiter.</returns>
    private static string ReadFromServer()
    {
        byte[] buffer = new byte[1024];
        StringBuilder responseBuilder = new StringBuilder();
        int bytesRead;
        string endDelimiter = "\0END\0";

        do
        {
            bytesRead = DynamoXICM.DynamoXICM.GetPipeServer.Read(buffer, 0, buffer.Length);
            responseBuilder.Append(Encoding.UTF8.GetString(buffer, 0, bytesRead));
        } while (!responseBuilder.ToString().Contains(endDelimiter));

        string response = responseBuilder.ToString();
        return response.Substring(0, response.IndexOf(endDelimiter));
    }

    /// <summary>
    /// Parses the JSON response string and extracts the response property.
    /// </summary>
    /// <param name="response">The JSON response string from the server.</param>
    /// <returns>The extracted response property as a string.</returns>
    private static string ParseResponse(string response)
    {
        JsonDocument json = JsonDocument.Parse(response);
        return json.RootElement.GetProperty("response").ToString();
    }
}

Error: (48,9): error CS0103: The name 'DynamoXICM' does not exist in the current context
(49,9): error CS0103: The name 'DynamoXICM' does not exist in the current context
(65,25): error CS0103: The name 'DynamoXICM' does not exist in the current context